# Obtencion de modelos

In [6]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from itertools import product
from itertools import chain
import statsmodels.api as sm


pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 

In [7]:
data = pd.read_excel("DatosEleccionesEspaña.xlsx", sheet_name="DatosEleccionesEspaña")

out_variables_dependientes = ["AbstencionAlta",
                              "AbstentionPtge",
                              "Izda_Pct",
                              "Dcha_Pct",
                              "Otros_Pct",
                              "Izquierda",
                              "Derecha"]

Y = data["AbstentionPtge"]
X = data.drop(columns=out_variables_dependientes)

## 2. Dividimos los datos en un conjunto de entrenamiento y otro de prueba

In [8]:
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size=0.2 , random_state =1234567)

In [9]:
def lm(varObjCont, datos, var_cont, var_categ, var_interac=[]):
    """
    Ajusta un modelo de regresión lineal a los datos y devuelve
    información relacionada con el modelo.

    Parámetros:
    varObjCont (Series o array): La variable objetivo continua que se está tratando de predecir.
    datos (DataFrame): DataFrame de datos que contiene las variables de entrada o predictoras.
    var_cont (lista): Lista de nombres de variables continuas.
    var_categ (lista): Lista de nombres de variables categóricas.
    var_interac (lista, opcional): Lista de pares de variables para la interacción (por defecto es una lista vacía).

    Returns:
    dict: Un diccionario que contiene información relacionada con el modelo ajustado, incluyendo el modelo en sí,
    las listas de variables continuas y categóricas, las variables de interacción (si se especifican)
    y el DataFrame X utilizado para realizar el modelo.
    """

    # Prepara los datos para el modelo, incluyendo la dummificación de variables categóricas y la creación de interacciones.
    datos = crear_data_modelo(datos, var_cont, var_categ, var_interac)

    # Ajusta un modelo de regresión lineal a los datos y almacena la información del modelo en 'Modelo'.
    output = {
        'Modelo': sm.OLS(varObjCont, sm.add_constant(datos)).fit(),
        'Variables': {
            'cont': var_cont,
            'categ': var_categ,
            'inter': var_interac
        },
        'X': datos
    }

    return output

In [10]:
def crear_data_modelo(datos, var_cont, var_categ, var_interac=[]):
    """
    Función para preparar datos para su uso en un modelo, incluyendo variables numéricas, categóricas y la interacción entre variables.

    Parámetros:
    datos (DataFrame): El DataFrame de datos original.
    var_cont (lista): Lista de nombres de variables continuas.
    var_categ (lista): Lista de nombres de variables categóricas.
    var_interac (lista, opcional): Lista de pares de variables para la interacción (por defecto es una lista vacía).

    Returns:
    datos_v: DataFrame preparado con variables categóricas codificadas y las interacciones calculadas.
    """

    # Verifica si no hay variables de entrada
    if len(var_cont + var_categ + var_interac) == 0:
        return datos[[]]

    # Selecciona las variables continuas y categóricas
    datos_v = datos[var_cont + var_categ]

    # Codifica las variables categóricas si existen
    if len(var_categ) > 0:
        datos_v = pd.get_dummies(datos_v, columns=var_categ, drop_first=True)

    # Calcula una lista de todas las variables de entrada
    variables_total = list(set(var_cont + var_categ + list(chain.from_iterable(var_interac))))

    # Identifica las nuevas variables categóricas
    var_categ_new = datos[variables_total].select_dtypes(include=['object', 'category']).columns.tolist()

    # Codifica las nuevas variables categóricas
    datos = pd.get_dummies(datos[variables_total], columns=var_categ_new, drop_first=True)

    # Calcula las interacciones si se especifican
    if len(var_interac) > 0:
        n_var = len(datos.columns)
        for interac in var_interac:
            vars1 = [x for x in datos.columns[:n_var] if interac[0] in x]
            vars2 = [x for x in datos.columns[:n_var] if interac[1] in x]
            combinaciones = list(product(vars1, vars2))
            for comb in combinaciones:
                datos[comb[0] + '_' + comb[1]] = datos[comb[0]] * datos[comb[1]]

        # Selecciona las variables de interacción calculadas
        datos_interac = datos.iloc[:, list(range(n_var, len(datos.columns)))]

        # Concatena las variables continuas, categóricas y las interacciones
        return pd.concat([datos_v, datos_interac], axis=1)

    # Si no hay interacciones, se devuelve el DataFrame con variables categóricas codificadas
    return datos_v


In [11]:
# Definir las variables
var_cont = ['X1', 'X2']
var_categ = ['X3', 'X4', 'X5']
var_interac = [('X4', 'X5')]

# Llamar a la función lm
modelo = lm(Y_train, X_train, var_cont, var_categ, var_interac)

KeyError: "None of [Index(['X1', 'X2', 'X3', 'X4', 'X5'], dtype='object')] are in the [columns]"

In [ ]:
# Definir las variables
var_cont = []
var_categ = ['Etiqueta', 'CalifProductor', 'Clasificacion']

# Llamar a la función lm
modelo = lm(y_train, x_train, var_cont, var_categ)

# Mostrar el resumen del modelo ajustado
modelo['Modelo'].summary()
